In [ ]:
#@title ## Set up the notebook
!pip install Biopython
from Bio import SeqIO
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
import seaborn as sns
from collections import Counter
from sklearn import model_selection, linear_model
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

# import gdown
# data_path = 'https://drive.google.com/uc?id=1f1CtRwSohB7uaAypn8iA4oqdXlD_xXL1'
# cov2_sequences = 'SARS_CoV_2_sequences_global.fasta'
# gdown.download(data_path, cov2_sequences, True)

!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20DNA%20Detectives/SARS_CoV_2_sequences_global.fasta'
cov2_sequences = 'SARS_CoV_2_sequences_global.fasta'



In [ ]:
#@title Load data from previous notebook
sequences = [r for r in SeqIO.parse(cov2_sequences, 'fasta')]
# This can take a couple minutes!
#@title ####Example Solution
# Note: This can take a couple minutes to run! 
# but we can monitor our progress using the tqdm library
mutation_df = pd.DataFrame()
n_bases_in_seq = len(sequences[0])

print("Creating feature matrix....")
# Iterate though all positions in this sequence.
for location in (range(n_bases_in_seq)): # tqdm is a nice library that prints our progress.
  bases_at_location = np.array([s[location] for s in sequences])
  # If there are no mutations at this position, move on.
  if len(set(bases_at_location))==1: continue # If
  for base in ['A', 'T', 'G', 'C', '-']:
    feature_values = (bases_at_location==base)
    
    
    # Set the values of any base that equals 'N' to np.nan.
    feature_values[bases_at_location=='N'
                   ] = np.nan
    
    # Convert from T/F to 0/1.
    feature_values  = feature_values*1
    
    # Make the column name look like <location>_<base> (1_A, 2_G, 3_A, etc.)
    column_name = str(location) + '_' + base
    mutation_df[column_name] = feature_values

print("Formatting labels....")
countries = [(s.description).split('|')[-1] for s in sequences]
countries_to_regions_dict = {
         'Australia': 'Oceania',
         'China': 'Asia',
         'Hong Kong': 'Asia',
         'India': 'Asia',
         'Nepal': 'Asia',
         'South Korea': 'Asia',
         'Sri Lanka': 'Asia',
         'Taiwan': 'Asia',
         'Thailand': 'Asia',
         'USA': 'North America',
         'Viet Nam': 'Asia'
}
regions = [countries_to_regions_dict[c] if c in 
           countries_to_regions_dict else 'NA' for c in countries]
mutation_df['label'] = regions

print("Balancing data labels....")
balanced_df = mutation_df.copy()
balanced_df['label'] = regions
balanced_df = balanced_df[balanced_df.label!='NA']
balanced_df = balanced_df.drop_duplicates()
samples_north_america = balanced_df[balanced_df.label== ####### FILL IN ####
                                    'North America']
samples_oceania = balanced_df[balanced_df.label== ##### FILL IN #########
                              'Oceania']
samples_asia = balanced_df[balanced_df.label== ##### FILL IN #######
                           'Asia']

# Number of samples we will use from each region.
n = min(len(samples_north_america),
        len(samples_oceania),
        len(samples_asia))

balanced_df = pd.concat([samples_north_america[:n],
                    samples_asia[:n],
                    samples_oceania[:n]])

X = balanced_df.drop('label', 1)
Y = balanced_df.label

In [ ]:
#@title #### Retrain previous model
lm = linear_model.LogisticRegression(
    multi_class="multinomial", max_iter=1000,
    fit_intercept=False, tol=0.001, solver='saga', random_state=42)

# Split into training/testing set.
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(
    X, Y, train_size=.8, random_state=1)

# Train/fit model.
lm.fit(X_train, Y_train)

In [ ]:
#@title ### **Exercise: Run the following to visualize our X dataset in principal component space.**
data = mutation_df#[variant_df.regions=='USA']
pca = decomposition.PCA(n_components=2)
pca.fit(X)
df = pd.DataFrame()
df['Principal Component 1'] = [pc[0] for pc in pca.transform(X)]
df['Principal Component 2'] = [pc[1] for pc in pca.transform(X)]
plt.figure(figsize=(5,5))
sns.scatterplot(data=df, x='Principal Component 1', y='Principal Component 2')
plt.show()

In [ ]:
#@title ### **Exercise: Color the data points by region**
df['color'] = Y
plt.figure(figsize=(5,5))
sns.scatterplot(data=df, x='Principal Component 1', y='Principal Component 2', hue='color')